In [1]:
!pip install -q seaborn

In [ ]:
# import all packages
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import tempfile
import os
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
print(tf.__version__)
# To make the results reproducible, set the random seed value.
tf.random.set_seed(22)
np.random.seed(42)

2.12.0


# Logistic regression with manually defined 0, 1 labels


In [ ]:
# upload dataset CSV file from local
from google.colab import files
uploaded = files.upload()

Saving discriptor.csv to discriptor (1).csv


In [ ]:
import io
# you can download the dataset file from shared drive
df = pd.read_csv(io.BytesIO(uploaded[next(iter(uploaded))]))

In [ ]:
# trim columns with NaN
nan_columns = df.columns[df.isna().any()].tolist()

# Delete empty columns
dataset = df.drop(nan_columns, axis=1)

In [ ]:
dataset

,Name,SMILES,Binary_Efficiency,Lipid_ratio,Chol_ratio,Helper_ratio,PEG-ratio,nAcid,ALogP,ALogp2,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,adhm201400039*1O14B,CN(C)CCCN(CCC(OCCSSCCCCCCCCCC)=O)CCC(OCCSSCCCC...,0,100.0,0.0,0.0,0.0,0.0,-2.1966,4.825052,...,6.177581,88.161076,1.959135,29.420440,11.022162,6.406387,12132.0,46.0,11.976,182.0
1,adhm201400039*1O14,CN(C)CCCN(CCC(OCCCCCCCCCCCCCC)=O)CCC(OCCCCCCCC...,0,100.0,0.0,0.0,0.0,0.0,-6.9094,47.739808,...,5.191839,88.161076,1.959135,17.428549,11.022162,6.406387,12132.0,46.0,14.076,182.0
2,adhm201400039*1O16B,CN(C)CCCN(CCC(OCCSSCCCCCCCCCCCC)=O)CCC(OCCSSCC...,1,100.0,0.0,0.0,0.0,0.0,-3.3486,11.213122,...,6.035311,96.161075,1.962471,29.421795,11.022228,6.406389,15888.0,50.0,14.252,198.0
3,adhm201400039*1O16,CN(C)CCCN(CCC(OCCCCCCCCCCCCCCCC)=O)CCC(OCCCCCC...,0,100.0,0.0,0.0,0.0,0.0,-8.0614,64.986170,...,5.145620,96.161075,1.962471,17.428617,11.022228,6.406389,15888.0,50.0,16.352,198.0
4,adhm201400039*1O18B,CN(C)CCCN(CCC(OCCSSCCCCCCCCCCCCCC)=O)CCC(OCCSS...,1,100.0,0.0,0.0,0.0,0.0,-4.5006,20.255400,...,5.917605,104.161074,1.965303,29.422134,11.022245,6.406390,20364.0,54.0,16.528,214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3418,srep22137*TNT-F23,CCCCCCCCCCC(O)CN(C)CCN1C(=O)N(CCN(C)CC(O)CCCCC...,0,30.0,40.0,30.0,0.0,0.0,-9.0525,81.947756,...,5.466277,117.923377,1.965390,35.270678,15.303981,19.966697,21675.0,78.0,12.498,264.0
3419,srep22137*TNT-F24,CCCCCCCCCCC(O)CN(C)CCN1C(=O)N(CCN(C)CC(O)CCCCC...,0,30.0,45.0,30.0,0.0,0.0,-9.0525,81.947756,...,5.466277,117.923377,1.965390,35.270678,15.303981,19.966697,21675.0,78.0,12.498,264.0
3420,srep22137*TNT-F25,CCCCCCCCCCC(O)CN(C)CCN1C(=O)N(CCN(C)CC(O)CCCCC...,1,30.0,35.0,40.0,0.0,0.0,-9.0525,81.947756,...,5.466277,117.923377,1.965390,35.270678,15.303981,19.966697,21675.0,78.0,12.498,264.0
3421,srep22137*TNT-F26,CCCCCCCCCCC(O)CN(C)CCN1C(=O)N(CCN(C)CC(O)CCCCC...,1,30.0,40.0,40.0,0.0,0.0,-9.0525,81.947756,...,5.466277,117.923377,1.965390,35.270678,15.303981,19.966697,21675.0,78.0,12.498,264.0


#Random split

In [ ]:


training_dataset, temp_dataset = train_test_split(dataset, test_size=0.20, random_state=1)


validation_dataset, testing_dataset = train_test_split(temp_dataset, test_size=0.50, random_state=1)


In [ ]:
testing_dataset

,Name,SMILES,Binary_Efficiency,Lipid_ratio,Chol_ratio,Helper_ratio,PEG-ratio,nAcid,ALogP,ALogp2,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
1653,adma.202006619*4A3&SC8-18,CCCCCCCCSCC(C)C(=O)OCCOC(=O)[CH]CN(C[CH]C(=O)O...,0,50.000000,40.000000,20.0,2.500000,0.0,-3.9105,15.292010,...,6.185248,192.213887,1.961366,65.965558,43.934795,10.100724,90153.0,116.0,18.933,416.0
2571,pnas.1121423109*LF99,CCCCCCCCCCCCCCOC(=O)CCN(CCC(=O)OCCCCCCCCCCCCCC...,1,73.668000,11.521806,0.0,14.810194,0.0,-13.7016,187.733843,...,5.245508,157.834282,1.972929,28.962594,22.053129,6.909465,51938.0,85.0,27.520,326.0
2409,adhm.201901487*A5,CCCCCCCCCCCCCC(O)CN(CCOCCN(CC(O)CCCCCCCCCCCCC)...,0,25.000000,20.000000,10.0,8.500000,0.0,-19.5988,384.112961,...,5.050746,200.294551,1.983114,35.993197,18.759310,17.233887,98416.0,113.0,30.725,424.0
1120,s41587*CB-6&R-21,CCCCC#CCOC(=O)OC(C/C=C\CCCCCCCCOC(=O)CCN(C)CCO...,1,35.000000,46.500000,16.0,2.500000,0.0,-3.2512,10.570301,...,5.759453,157.469358,1.968367,40.521404,34.110145,6.411258,70144.0,87.0,19.448,330.0
397,nbt1402*O14&A61,CCCCCCCCCCCCCCOC(=O)CCN(CC)CCN(CC)CCC(=O)OCCCC...,0,82.934153,7.471145,0.0,9.594702,0.0,-6.1206,37.461744,...,5.179459,90.151289,1.959811,17.666550,11.006111,6.660439,13521.0,49.0,14.564,186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,s41587*CB-8&R-32,CCCCCCC#CCOC(=O)OC(C/C=C\CCCCCCCCOC(=O)CCN(CCC...,0,35.000000,46.500000,16.0,2.500000,0.0,-2.2786,5.192018,...,5.488780,95.853638,1.956197,23.461003,14.084439,9.376565,15100.0,52.0,11.579,202.0
623,nbt1402*N14&A100,CCCCCCCCCCCCCCNC(=O)CCN(CCCN(CCC(=O)NCCCCCCCCC...,1,73.505748,11.598744,0.0,14.895508,0.0,-16.1644,261.287827,...,5.126050,159.833017,1.973247,28.945690,10.116423,18.829267,54396.0,86.0,25.754,330.0
576,nbt1402*N13&A110,CCCCCCCCCCCCCNC(=O)CCN(CCC(=O)NCCCCCCCCCCCCC)C...,1,65.565958,15.074652,0.0,19.359389,0.0,-22.0386,485.699890,...,5.172660,233.205747,1.976320,46.947816,15.177601,31.770214,127200.0,129.0,34.428,486.0
2064,advs.202301929*Lipid 23,CCCCCCCCC(CCCCCC)C(=O)OCCCCCCCCN(CCCCCCCCCC(=O...,1,40.000000,48.500000,10.0,1.500000,0.0,-9.1738,84.158606,...,5.221174,123.703101,1.963541,23.061927,16.646087,6.415840,28672.0,70.0,19.472,260.0


In [ ]:
x_train, y_train = training_dataset.iloc[:, 3:], training_dataset.iloc[:, 2]
x_validation, y_validation = validation_dataset.iloc[:, 3:], validation_dataset.iloc[:, 2]
x_test, y_test = testing_dataset.iloc[:, 3:], testing_dataset.iloc[:, 2]

In [ ]:
from collections import defaultdict
x_dic = defaultdict(list)
y_dic = defaultdict(list)
all_doi = set()
for i in range(len(validation_dataset)):
  full_name = validation_dataset.iloc[i, 0]
  doi = full_name.split('*')[0]
  all_doi.add(doi)
  x = x_validation.iloc[i, :]
  y = y_validation.iloc[i]
  x_dic[doi].append(x)
  y_dic[doi].append(y)
print(len(validation_dataset), len(all_doi))

342 37


In [ ]:
# Define the model architecture
from keras.layers import BatchNormalization
from keras.layers import LayerNormalization
from tensorflow.keras.regularizers import l2

model = tf.keras.models.Sequential([
    BatchNormalization(),
    LayerNormalization(),
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(1448,), kernel_regularizer=l2(0.03)), # L2
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=l2(0.03)), # L2
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.03)), # L2
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.03)), # L2
    tf.keras.layers.Dense(1, activation='sigmoid',kernel_regularizer=l2(0.03))
])


In [ ]:
# Compile the model with logistic loss
optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              #loss=precision_loss,
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision()])

In [ ]:
from keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=400, batch_size=256, validation_data=(x_validation, y_validation), callbacks=[early_stopping])

Epoch 1/400
11/11 [==============================] - 2s 92ms/step - loss: 55.7095 - accuracy: 0.7447 - precision_22: 0.2302 - val_loss: 53.8289 - val_accuracy: 0.8012 - val_precision_22: 1.0000
Epoch 2/400
11/11 [==============================] - 1s 69ms/step - loss: 52.5161 - accuracy: 0.7706 - precision_22: 0.8000 - val_loss: 50.7463 - val_accuracy: 0.8012 - val_precision_22: 1.0000
Epoch 3/400
11/11 [==============================] - 1s 70ms/step - loss: 49.4749 - accuracy: 0.7703 - precision_22: 0.5833 - val_loss: 47.7997 - val_accuracy: 0.8070 - val_precision_22: 0.8000
Epoch 4/400
11/11 [==============================] - 1s 65ms/step - loss: 46.5827 - accuracy: 0.7695 - precision_22: 0.5000 - val_loss: 44.9794 - val_accuracy: 0.8070 - val_precision_22: 0.8000
Epoch 5/400
11/11 [==============================] - 1s 66ms/step - loss: 43.8451 - accuracy: 0.7703 - precision_22: 0.5625 - val_loss: 42.3450 - val_accuracy: 0.8070 - val_precision_22: 0.8000
Epoch 6/400
11/11 [===========

In [ ]:
# Evaluate the model
validation_loss, validation_acc, validation_precision = model.evaluate(x_validation, y_validation)

11/11 [==============================] - 0s 6ms/step - loss: 0.5108 - accuracy: 0.8041 - precision_22: 0.7500


In [ ]:
test_loss, test_acc, test_precision = model.evaluate(x_test, y_test)

11/11 [==============================] - 0s 6ms/step - loss: 0.5350 - accuracy: 0.7726 - precision_22: 0.5000


In [ ]:
y_validation

14      0
15      0
18      0
20      1
25      0
       ..
3400    1
3402    0
3406    0
3411    1
3415    0
Name: Binary_Efficiency, Length: 856, dtype: int64

In [ ]:
# check the precision in each paper
i = 1
result_dic = {}
for doi in all_doi:
  x = pd.DataFrame(x_dic[doi])
  y = pd.DataFrame(y_dic[doi])
  print(i, doi)
  validation_loss, validation_acc, validation_precision = model.evaluate(x, pd.DataFrame(y))
  y_pred = model.predict(x)
  true_positives = tf.math.count_nonzero(tf.logical_and(tf.equal(y, 1), tf.equal(tf.round(y_pred), 1)), axis=0)
  predicted_positives = tf.math.count_nonzero(tf.equal(tf.round(y_pred), 1), axis=0)
  total_positives = tf.math.count_nonzero(tf.equal(y, 1), axis=0)
  print(true_positives/predicted_positives)
  print("true_positive: ", true_positives)
  print("predicted_positive: ", predicted_positives)
  print("total_positives: ", total_positives)
  result_dic[doi] = (validation_precision, true_positives, predicted_positives, total_positives)
  i += 1

1 j.omtn.2020.01.018
1/1 [==============================] - 0s 167ms/step
tf.Tensor([0.], shape=(1,), dtype=float64)
true_positive:  tf.Tensor([0], shape=(1,), dtype=int64)
predicted_positive:  tf.Tensor([2], shape=(1,), dtype=int64)
total_positives:  tf.Tensor([0], shape=(1,), dtype=int64)
2 acs.nanolett.5b02497
1/1 [==============================] - 0s 27ms/step
tf.Tensor([0.], shape=(1,), dtype=float64)
true_positive:  tf.Tensor([0], shape=(1,), dtype=int64)
predicted_positive:  tf.Tensor([6], shape=(1,), dtype=int64)
total_positives:  tf.Tensor([2], shape=(1,), dtype=int64)
3 s41587
6/6 [==============================] - 0s 8ms/step
tf.Tensor([1.], shape=(1,), dtype=float64)
true_positive:  tf.Tensor([3], shape=(1,), dtype=int64)
predicted_positive:  tf.Tensor([3], shape=(1,), dtype=int64)
total_positives:  tf.Tensor([16], shape=(1,), dtype=int64)
4 jacs.2c02706
1/1 [==============================] - 0s 34ms/step
tf.Tensor([nan], shape=(1,), dtype=float64)
true_positive:  tf.Tensor

In [ ]:
for doi in result_dic:
  print(doi)
  for x in result_dic[doi]:
   tf.print(x)


bc1000041r
1.0
[2]
[2]
[6]
j.omtn.2020.01.018
0.0
[0]
[1]
[0]
acs.nanolett.5b02497
0.2222222238779068
[2]
[9]
[2]
j.ymthe.2018.03.010
0.0
[0]
[1]
[0]
jacs.2c02706
0.0
[0]
[0]
[4]
anie.201610209
0.625
[5]
[8]
[8]
adma.201805308
0.800000011920929
[4]
[5]
[4]
j.omtn.2022.09.017
0.0
[0]
[0]
[2]
acs.nanolett.9b04246
0.0
[0]
[0]
[2]
s12195-016-0436-9
0.0
[0]
[0]
[0]
acs.nanolett.2c04883
1.0
[2]
[2]
[2]
adma.202006619
0.0
[0]
[0]
[7]
srep22137
0.0
[0]
[0]
[2]
bc0c00295
0.0
[0]
[0]
[4]
j.biomaterials.2022.121645
0.0
[0]
[0]
[1]
ja301621z
0.7222222089767456
[13]
[18]
[15]
j.actbio.2022.09.065
0.0
[0]
[0]
[0]
j.jconrel.2019.10.028
1.0
[1]
[1]
[1]
s41565-019-0600-1
1.0
[1]
[1]
[7]
acsnano.2c07822
0.0
[0]
[1]
[3]
adma.202209624
1.0
[2]
[2]
[4]
s41587
0.3333333432674408
[1]
[3]
[16]
sciadv.abc2315
0.0
[0]
[0]
[1]
j.jconrel.2020.08.031
0.0
[0]
[0]
[1]
adhm.201901487
0.0
[0]
[1]
[5]
bc200572w
1.0
[2]
[2]
[4]
pnas.0910603106
0.5714285969734192
[4]
[7]
[6]
sb300023h
0.0
[0]
[1]
[2]
nbt1402
0.5625
[9]
[

In [ ]:
for x in result_dic['ja301621z']:
   tf.print(x)

0.7222222089767456
[13]
[18]
[15]


In [ ]:
y_pred = tf.squeeze(model.predict(x_validation))
true_positives = tf.math.count_nonzero(tf.logical_and(tf.equal(y_validation, 1), tf.equal(tf.round(y_pred), 1)), axis=0)
predicted_positives = tf.math.count_nonzero(tf.equal(tf.round(y_pred), 1), axis=0)
print(true_positives/predicted_positives)
print("true_positive: ", true_positives)
print("predicted_positive: ", predicted_positives)

27/27 [==============================] - 0s 9ms/step
tf.Tensor(0.5766423357664233, shape=(), dtype=float64)
true_positive:  tf.Tensor(79, shape=(), dtype=int64)
predicted_positive:  tf.Tensor(137, shape=(), dtype=int64)


## Predictions on LongChuan data

In [ ]:
# upload dataset CSV file from local
from google.colab import files
uploaded = files.upload()

Saving longchuan1_processed.csv to longchuan1_processed (2).csv


In [ ]:
import io
# you can download the dataset file from shared drive
test_dataset = pd.read_csv(io.BytesIO(uploaded[next(iter(uploaded))]))

In [ ]:
test_dataset

,Name,SMILES,Lipid_ratio,Chol_ratio,Helper_ratio,PEG-ratio,nAcid,ALogP,ALogp2,AMR,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,lcsw030,CCCCC(COC(CCCCCC)=O)N(C(COC(CCCCCC)=O)CCCC)C,50,38.5,10,1.5,0,-3.9015,15.221702,110.8539,...,0.893533,0.059180,0.397080,0.219313,0.279019,46.308873,210.01223,504.644602,0.84030,0.895413
1,lcsw031,CCCCC(COC(C)=O)N(C(COC(C)=O)CCCC)C,50,38.5,10,1.5,0,-1.7499,3.062150,77.5141,...,0.653886,0.189789,0.358368,0.300343,0.430861,16.642408,70.90083,176.966171,0.48083,1.089572
2,lcsw034,CCCCC(COC(CCCCCCCCCCCCCCCCC)=O)N(C(COC(CCCCCCC...,50,38.5,10,1.5,0,-10.2375,104.806406,174.9091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,lcsm002,CN(CCN(CCC(OCC[N+](C)(CC(OCCCCCCCCCCCCCCCCCC)=...,50,38.5,10,1.5,0,-24.2069,585.974008,450.9239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,lcsm003,OCCN(CCSCCC(OCCCCCCCCCCCCCCCCCC)=O)CCSCCC(OCCC...,50,38.5,10,1.5,0,-8.3759,70.155701,194.8220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,1-1-C14,OCCN(CCN(CCSCCC(OCCCCCCCCCCCCCC)=O)CCSCCC(OCCC...,50,38.5,10,1.5,0,-8.8595,78.490740,262.1727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,1-1-C15,OCCN(CCN(CCSCCC(OCCCCCCCCCCCCCCC)=O)CCSCCC(OCC...,50,38.5,10,1.5,0,-9.7235,94.546452,270.9075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,1-1-C16,OCCN(CCN(CCSCCC(OCCCCCCCCCCCCCCCC)=O)CCSCCC(OC...,50,38.5,10,1.5,0,-10.5875,112.095156,279.6423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,1-1-C17,OCCN(CCN(CCSCCC(OCCCCCCCCCCCCCCCCC)=O)CCSCCC(O...,50,38.5,10,1.5,0,-11.4515,131.136852,288.3771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_dataset_trim = test_dataset.drop(nan_columns[2:], axis=1)

In [ ]:
test_dataset_trim

,Name,SMILES,Lipid_ratio,Chol_ratio,Helper_ratio,PEG-ratio,nAcid,ALogP,ALogp2,AMR,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,lcsw030,CCCCC(COC(CCCCCC)=O)N(C(COC(CCCCCC)=O)CCCC)C,50,38.5,10,1.5,0,-3.9015,15.221702,110.8539,...,5.357617,62.000478,1.937515,14.365850,11.056606,3.309244,3583,38,9.095,132
1,lcsw031,CCCCC(COC(C)=O)N(C(COC(C)=O)CCCC)C,50,38.5,10,1.5,0,-1.7499,3.062150,77.5141,...,5.731654,42.020450,1.910020,13.895224,10.598851,3.296373,1153,26,4.037,92
2,lcsw034,CCCCC(COC(CCCCCCCCCCCCCCCCC)=O)N(C(COC(CCCCCCC...,50,38.5,10,1.5,0,-10.2375,104.806406,174.9091,...,5.057892,105.999836,1.962960,14.381024,11.071365,3.309659,19665,60,21.613,220
3,lcsm002,CN(CCN(CCC(OCC[N+](C)(CC(OCCCCCCCCCCCCCCCCCC)=...,50,38.5,10,1.5,0,-24.2069,585.974008,450.9239,...,5.389095,267.247813,1.965057,67.640718,43.910652,23.730066,228643,154,33.937,584
4,lcsm003,OCCN(CCSCCC(OCCCCCCCCCCCCCCCCCC)=O)CCSCCC(OCCC...,50,38.5,10,1.5,0,-8.3759,70.155701,194.8220,...,5.494471,110.320655,1.970012,22.840263,13.437942,3.388544,25902,57,19.650,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,1-1-C14,OCCN(CCN(CCSCCC(OCCCCCCCCCCCCCC)=O)CCSCCC(OCCC...,50,38.5,10,1.5,0,-8.8595,78.490740,262.1727,...,5.641584,144.005910,1.972684,34.839880,18.940400,6.862910,45650,77,22.823,296
184,1-1-C15,OCCN(CCN(CCSCCC(OCCCCCCCCCCCCCCC)=O)CCSCCC(OCC...,50,38.5,10,1.5,0,-9.7235,94.546452,270.9075,...,5.598380,150.005908,1.973762,34.839948,18.940466,6.862911,51644,80,24.530,308
185,1-1-C16,OCCN(CCN(CCSCCC(OCCCCCCCCCCCCCCCC)=O)CCSCCC(OC...,50,38.5,10,1.5,0,-10.5875,112.095156,279.6423,...,5.558861,156.005908,1.974758,34.839983,18.940499,6.862911,58148,83,26.237,320
186,1-1-C17,OCCN(CCN(CCSCCC(OCCCCCCCCCCCCCCCCC)=O)CCSCCC(O...,50,38.5,10,1.5,0,-11.4515,131.136852,288.3771,...,5.522576,162.005907,1.975682,34.840000,18.940516,6.862911,65183,86,27.944,332


In [ ]:
# Evaluate the model
predict = model.predict(test_dataset_trim.iloc[:, 2:])

6/6 [==============================] - 0s 8ms/step


In [ ]:
tf.math.count_nonzero(tf.equal(tf.round(predict), 1))

<tf.Tensor: shape=(), dtype=int64, numpy=105>

In [ ]:
predict[:]

array([[0.0076154 ],
       [0.02015227],
       [0.01392802],
       [0.00811152],
       [0.06375402],
       [0.02222195],
       [0.12362438],
       [0.14221251],
       [0.17224817],
       [0.09925922],
       [0.12396469],
       [0.15387851],
       [0.18768913],
       [0.22619726],
       [0.2689499 ],
       [0.3161805 ],
       [0.37229478],
       [0.433293  ],
       [0.48227414],
       [0.09726673],
       [0.12399327],
       [0.15361682],
       [0.19123769],
       [0.23427038],
       [0.28813076],
       [0.35138512],
       [0.4173532 ],
       [0.48996255],
       [0.5583714 ],
       [0.60209936],
       [0.6349884 ],
       [0.6608901 ],
       [0.21112074],
       [0.2605301 ],
       [0.30660152],
       [0.3644028 ],
       [0.42744017],
       [0.49173847],
       [0.5498749 ],
       [0.5914297 ],
       [0.625364  ],
       [0.64809364],
       [0.6547926 ],
       [0.6579412 ],
       [0.66155785],
       [0.36789843],
       [0.42289966],
       [0.473

In [ ]:
import numpy as np

def save_array_to_csv(array, filename, download=False):
    array = np.array(array).reshape(-1, 1)
    df = pd.DataFrame(array, columns=['Values'])
    df.to_csv(filename, index=False)
    files.download(filename)


save_array_to_csv(predict[:], 'array.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>